In [24]:
import tensorflow as tf

In [25]:
(x_train, y_train),(x_test, y_test)= tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train,x_test = x_train.reshape([-1,784]),x_test.reshape([-1,784])
x_train,x_test = x_train/255,x_test/255.
y_train, y_test = tf.one_hot(y_train, depth=10),tf.one_hot(y_test, depth=10)

In [26]:
learning_rate=0.001
num_epochs =30
batch_size= 256
display_stpe =1
input_size =784
hidden1_size =256
hidden2_size =256
output_size =10

In [27]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data =  train_data.shuffle(60000).batch(batch_size)

def random_normal_intializer_with_stddev_1():
    return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

In [28]:
class ANN(tf.keras.Model):
    def __init__(self):
       super(ANN, self).__init__()
       self.hidden_layer_1 = tf.keras.layers.Dense(hidden1_size,
                                                activation='relu',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
       self.hidden_layer_2 = tf.keras.layers.Dense(hidden2_size,
                                                activation='relu',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())
       self.output_layer = tf.keras.layers.Dense(output_size,
                                              activation=None,
                                              kernel_initializer=random_normal_intializer_with_stddev_1(),
                                              bias_initializer=random_normal_intializer_with_stddev_1())
       
       def call(self, x):
         H1_output = self.hidden_layer_1(x)
         H2_output = self.hidden_layer_2(H1_output)
         logits = self.output_layer(H2_output)

         return logits


In [29]:

# cross-entropy 손실 함수를 정의합니다.
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# 최적화를 위한 Adam 옵티마이저를 정의합니다.
optimizer = tf.optimizers.Adam(learning_rate)

In [30]:
@tf.function
def train_step(model,x,y):
    with tf.GradientTape() as tape:
        y_prd = model(x)
        loss = cross_entropy_loss(y_prd,y)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [31]:
@tf.function
def compute_accuracy(y_prd, y):
    correct_prediction = tf.equal(tf.argmax(y_prd,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy

In [32]:
ANN_model = ANN()

In [34]:
for epoch in range(num_epochs):
  average_loss = 0.
  total_batch = int(x_train.shape[0] / batch_size)
  # 모든 배치들에 대해서 최적화를 수행합니다.
  for batch_x, batch_y in train_data:
    # 옵티마이저를 실행해서 파라마터들을 업데이트합니다.
    _, current_loss = train_step(ANN_model, batch_x, batch_y), cross_entropy_loss(ANN_model(batch_x), batch_y)
    # 평균 손실을 측정합니다.
    average_loss += current_loss / total_batch
  # 지정된 epoch마다 학습결과를 출력합니다.
  if epoch % display_step == 0:
    print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), average_loss))

# 테스트 데이터를 이용해서 학습된 모델이 얼마나 정확한지 정확도를 출력합니다.
print("정확도(Accuracy): %f" % compute_accuracy(ANN_model(x_test), y_test)) # 정확도: 약 94%

NotImplementedError: in user code:

    File "/var/folders/by/m4xy631x2snc43fg1lswdxd00000gn/T/ipykernel_59571/3542757828.py", line 4, in train_step  *
        y_prd = model(x)
    File "/Users/nicolekan/anaconda3/envs/AI/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/Users/nicolekan/anaconda3/envs/AI/lib/python3.8/site-packages/keras/engine/training.py", line 584, in call
        raise NotImplementedError(

    NotImplementedError: Exception encountered when calling layer "ann_2" "                 f"(type ANN).
    
    Unimplemented `tf.keras.Model.call()`: if you intend to create a `Model` with the Functional API, please provide `inputs` and `outputs` arguments. Otherwise, subclass `Model` with an overridden `call()` method.
    
    Call arguments received by layer "ann_2" "                 f"(type ANN):
      • inputs=tf.Tensor(shape=(256, 784), dtype=float32)
      • train